In [1]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"

# train test split paths
train_path = data_path / 'binary_tts/train'
test_path = data_path / 'binary_tts/test'
val_path = data_path / 'binary_tts/val'

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# silence max image size warning
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 

# import modeling packages
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# modeling metrics
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

changes to data generators:
    
    train batch_size 20
    
    test batch_size 50 -> 20
    
    val batch_size 109 -> 20
    
    all class_mode 'categorical' -> 'binary'

In [2]:
train_generator = ImageDataGenerator().flow_from_directory(str(train_path),
                                                           target_size=(150, 150),
                                                           batch_size=20,
                                                           class_mode='binary')
test_generator = ImageDataGenerator().flow_from_directory(str(test_path),
                                                          target_size=(150, 150),
                                                          batch_size=20,
                                                          class_mode='binary',
                                                          shuffle=False)
val_generator = ImageDataGenerator().flow_from_directory(str(val_path),
                                                          target_size=(150, 150),
                                                          batch_size=20,
                                                          class_mode='binary',
                                                          shuffle=False)

Found 6000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


changes to model architecture:
        
    output num_nodes 5 -> 2
    
    loss 'categorical_crossentropy' -> 'binary_crossentropy'

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(35, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(30, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 35)      980       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 35)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 30)        9480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 30)        0         
_________________________________________________________________
flatten (Flatten)            (None, 38880)             0         
_________________________________________________________________
dense (Dense)                (None, 40)                1555240   
_________________________________________________________________
dropout (Dropout)            (None, 40)                0

changes to fit:
    
    epochs 10 -> 5
    
    steps_per_epoch 200 -> 300

In [5]:
model.fit(
        train_generator,
        steps_per_epoch=300,
        epochs=5,
        validation_data=test_generator,
        validation_steps=10)

Epoch 1/5
278/300 [==========================>...] - ETA: 52s - loss: 7.6614 - accuracy: 0.5050

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


300/300 [==============================] - 738s 2s/step - loss: 7.6617 - accuracy: 0.5018 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 2/5
300/300 [==============================] - 734s 2s/step - loss: 7.6685 - accuracy: 0.4993 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 3/5
300/300 [==============================] - 737s 2s/step - loss: 7.6662 - accuracy: 0.4992 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 4/5
  2/300 [..............................] - ETA: 7:18 - loss: 7.6664 - accuracy: 0.4750

KeyboardInterrupt: 

In [ ]:
conf_mat = confusion_matrix(test_generator.classes, np.argmax(model.predict(test_generator), axis=1))

class_names = list(test_generator.class_indices.keys())

fig, ax = plt.subplots(figsize=(8,6))
model_7_path = pathlib.Path().absolute().parent.parent / 'reports/figures/mod_9_heatmap.png'
ax.set_ylim([0,1])
sns.heatmap(conf_mat, xticklabels=class_names, yticklabels=class_names, ax=ax, square=True, annot=True, cmap='Blues')
plt.savefig(model_7_path)

In [ ]:
accuracies = []
for i in range(0, 5):
    acc = conf_mat[i][i] / sum(conf_mat[i])
    accuracies.append(acc)
accuracies